In [1]:
import openai
from openai import OpenAI

import os

In [4]:
import requests

In [12]:
from elasticsearch import Elasticsearch

In [11]:
from tqdm.auto import tqdm

### Import ElasticSearch

In [2]:
es_client = Elasticsearch('http://localhost:9200')

In [3]:
es_client.info()

ObjectApiResponse({'name': 'eaf3ebfda6ef', 'cluster_name': 'docker-cluster', 'cluster_uuid': '-0gyrV-LQvCwgOsZggI2YA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Get data

In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Indexing

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:29<00:00, 32.65it/s]


### Quering

In [46]:
query = "How do I execute a command in a running docker container?"


search_query_1 = {
    "size": 10,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

search_query_2 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [41]:
response_1 = es_client.search(index=index_name, body=search_query_1)

In [42]:
response_1["hits"]["hits"][0]["_score"]

84.050095

In [47]:
response_2 = es_client.search(index=index_name, body=search_query_2)

In [54]:
response_2["hits"]["hits"][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Building a prompt

In [55]:
context_template = """
                Q: {question}
                A: {text}
""".strip()

In [71]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [65]:
context = ""

for doc in response_2["hits"]["hits"]:
    context = context + f"Q: {doc['_source']['question']}\nA: {doc['_source']['text']}\n\n"

In [72]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [73]:
len(prompt_template.format(question=query, context=context).strip())

1462

### Tokens

In [77]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [82]:
len(encoding.encode(text = prompt_template.format(question=query, context=context).strip()))

322

### OpenAI usage

In [3]:
client = OpenAI()

In [4]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
) 

In [5]:
response.choices[0].message.content

"Whether it's too late to join a course depends on several factors, such as the specific registration deadlines set by the educational institution or course provider, the course start date, and the nature of the course (self-paced vs. instructor-led).\n\nHere are some steps you can take to determine if you can still join:\n\n1. **Check the Registration Deadline**: Look up the official registration dates for the course. This information is usually available on the course website or through the institution's registrar's office.\n\n2. **Contact the Course Provider**: Reach out directly to the admissions office, course instructor, or another relevant contact at the educational institution. They may offer you some flexibility or provide options for late enrollment.\n\n3. **Consider Self-Paced Courses**: If the course is self-paced, there may be no strict deadlines, allowing you to start at any time.\n\n4. **Review the Syllabus**: Understanding the course content and requirements can help yo

In [ ]:
]

In [ ]:
[